# Impor Beberapa Package

In [ ]:
from netCDF4 import Dataset
import wrf

# Direktori WRF

In [ ]:
wrf_path = '/home/absen/WRF-Model/WRF/test/em_real'

# Buka File

In [ ]:
wrf_d03 = Dataset(f"{wrf_path}/wrfout_d03_2022-01-01_00:00:00")
wrf_d03

In [ ]:
wrf_t2 = wrf.getvar(wrf_d03, "T2", timeidx=wrf.ALL_TIMES)
wrf_t2

In [ ]:
lats, lons = wrf.latlon_coords(wrf_t2)